# Concur P-Card Spending Analysis

This notebook contains basic cleaning operations on a sample of the Concur P-Spend Data. We hope that in cleaning, we will be able to group transactions by supplier to better represent spending by supplier.

## Imports

In [1]:
import pandas as pd
import numpy as np
import openpyxl

## Downloading the Data

**Note**: We will be working with only the description column from the data set, as I wasn't able to pull the entire dataset from Oracle without surpassing the data limit.

In [2]:
oracle = pd.read_excel('concur_data.xlsx')

## Basic Cleaning

**Shape:** Before we begin basic cleaning, lets take a look at how many rows our data seems to have.

In [3]:
oracle.shape

(581548, 20)

**Correcting Columns:** Next, we can take this data and correct all the column names so that they reflect what is presented in oracle and on the databases.

In [4]:
oracle.columns = ['Invoice ID',
                   'Invoice Number', 
                   'Invoice Source', 
                   'Description', 
                   'Invoice Distribution Amount', 
                   'Invoice Creation Date',
                   'Supplier Invoice Date',
                   'Payment Date',
                   'Payment Method Code',
                   'Payment Number',
                   'Supplier Name',
                   'Financial Unit',
                   'Financial Unit Number',
                   'Financial Unit Number Level 2',
                   'Financial Unit Name Level 2',
                   'Financial Unit Number Level 3',
                   'Financial Unit Name Level 3',
                   'Financial Unit Number Level 4',
                   'Financial Unit Name Level 4',
                   'Payment Year']

**What does our data look like?** Lets sample ten rows of our data to see what it looks like

In [5]:
oracle.sample(10)

,Invoice ID,Invoice Number,Invoice Source,Description,Invoice Distribution Amount,Invoice Creation Date,Supplier Invoice Date,Payment Date,Payment Method Code,Payment Number,Supplier Name,Financial Unit,Financial Unit Number,Financial Unit Number Level 2,Financial Unit Name Level 2,Financial Unit Number Level 3,Financial Unit Name Level 3,Financial Unit Number Level 4,Financial Unit Name Level 4,Payment Year
194082,3891982,TE_2B0D026622054DB59313_T_BeckmeyerHillary,UCSD Concur,Airfare||Southwest,336.34,2022-12-10T05:15:57.000+00:00,2022-06-04T00:00:00.000+00:00,2022-12-11T00:00:00.000+00:00,TC,679738,Concur Campus Card Clearing,CASPO,2000020,200000A,Marine Science,200100B,Marine Sciences Research,200104C,Ocean and Atmosphere,2022
550448,6656326,TE_88EE2024BF8D43B29FEF_P_TruongJennifer,UCSD Concur,522600 - Fed Chargeable - Subscriptions and Me...,323.88,2024-07-05T02:37:13.000+00:00,2024-06-27T00:00:00.000+00:00,2024-07-06T00:00:00.000+00:00,EC,1115068,Concur Campus Card Clearing,Computer Science and Engineering,1000177,100000A,Academic Affairs,100800B,Dean of Engineering,100803C,Computer Science and Engineering,2024
192137,3882975,TE_730B0E1A123E48058277_T_DeanJon,UCSD Concur,Train/Bus||ACCESRAI L 1778878907,72,2022-12-08T04:16:59.000+00:00,2022-09-02T00:00:00.000+00:00,2022-12-09T00:00:00.000+00:00,TC,679058,Concur Campus Card Clearing,HS Anesthesiology,3000008,300000A,Health Science,300200B,School of Medicine,300202C,Anesthesiology,2022
119784,3272996,TE_ABE422DCBE624847BE2C_P_Garcia SevillanoCarmen,UCSD Concur,"Food, Ships Operation||RALPHS #0108",107.72,2022-07-06T03:05:58.000+00:00,2022-05-19T00:00:00.000+00:00,2022-07-07T00:00:00.000+00:00,EC,575619,Concur Campus Card Clearing,School of Public Health Dept,3000191,300000A,Health Science,300700B,School of Public Health,300700C,School of Public Health,2022
303385,4941565,TE_764F6349E5584BB689B3_P_SalinasFrancisco,UCSD Concur,Office Supplies||436486629,55.99,2023-06-14T03:41:27.000+00:00,2023-01-21T00:00:00.000+00:00,2023-06-15T00:00:00.000+00:00,EC,811006,Concur Campus Card Clearing,Early Academic Outreach Programs,1000153,100000A,Academic Affairs,100600B,Dean of Social Sciences,100606C,Ctr Res Educ Equity Asses/Teach Exc,2023
138912,3423774,TE_881BA1B4864A46CF907B_P_DereusJeffrey,UCSD Concur,Office Supplies||UC SAN DIEGO BOOKSTORE,31.23,2022-08-23T03:06:00.000+00:00,2022-06-30T00:00:00.000+00:00,2022-08-24T00:00:00.000+00:00,EC,599241,Concur Campus Card Clearing,Pediatrics Host Microbe Systems and Therapeutics,3000062,300000A,Health Science,300200B,School of Medicine,300209C,Pediatrics,2022
238026,4314680,TE_CFDE4E2BD4284DFDB8A5_T_HobanBryan,UCSD Concur,Hotel Tax||THE VILLAGE,149.67,2023-03-10T04:06:18.000+00:00,2022-10-17T00:00:00.000+00:00,2023-03-11T00:00:00.000+00:00,TC,741915,Concur Campus Card Clearing,Geosciences Research,2000016,200000A,Marine Science,200100B,Marine Sciences Research,200103C,Earth Sciences,2023
359184,5409415,TE_0F919D1722E743168A9F_T_GnannJeffrey,UCSD Concur,Airfare||United Airlines,155.24,2023-09-21T04:09:46.000+00:00,2023-08-15T00:00:00.000+00:00,2023-09-22T00:00:00.000+00:00,TC,868997,Concur Campus Card Clearing,Utility Enterprise,7000015,700000A,Resource Management and Planning,700300B,Facilities Management,700307C,Utility Enterprise,2023
302696,4939471,TE_B037C655AA3F4C0D9870_T_CederquistLynette,UCSD Concur,Hotel Tax||THE JUPITER,52.35,2023-06-13T22:09:25.000+00:00,2023-02-16T00:00:00.000+00:00,2023-06-14T00:00:00.000+00:00,TC,810566,Concur Campus Card Clearing,Balance Sheet Accounts only,4200100,420000A,CEO of Medical Center,421000B,CFO,421000C,Institutional Expense,2023
361075,5433880,TE_DF6500FB48514C4AB887_P_CoufalNicole,UCSD Concur,Lab Supplies||UVIC ACCT SERVICES ECOMM,1628.28,2023-09-23T03:12:39.000+00:00,2023-09-12T00:00:00.000+00:00,2023-09-24T00:00:00.000+00:00,EC,869818,Concur Campus Card Clearing,Pediatrics Allergy/Immunology/Rheumatology,3000048,300000A,Health Science,300200B,School of Medicine,300209C,Pediatrics,2023


**Getting the entries we want**: Oracle has a tendency to export the title of invoices as a row under the invoice to avoid these entries, since they are generally incorrectly formatted, we will filter out any row that has an invoice distribution number of 0.

In [7]:
oracle = oracle.drop(index = 0)
oracle = oracle.reset_index(drop = True)
oracle_clean = oracle.dropna(subset=['Description'])
oracle = oracle_clean[oracle_clean['Invoice Distribution Amount'] != 0]

**Inconsistencies:** We are looking at the description column to identify suppliers for each row. Lets see how many of the description columns deviate from the format "Supplier || Description".

In [8]:
oracle_incorrect = oracle_clean[~oracle_clean['Description'].str.contains('\|\|')]
oracle_incorrect.sample(10)

,Invoice ID,Invoice Number,Invoice Source,Description,Invoice Distribution Amount,Invoice Creation Date,Supplier Invoice Date,Payment Date,Payment Method Code,Payment Number,Supplier Name,Financial Unit,Financial Unit Number,Financial Unit Number Level 2,Financial Unit Name Level 2,Financial Unit Number Level 3,Financial Unit Name Level 3,Financial Unit Number Level 4,Financial Unit Name Level 4,Payment Year
334691,5231127,TE_GHM4_G_WilsonErickson,UCSD Concur,Gift Cards - HS Payments,5200,2023-08-05T04:10:48.000+00:00,2023-08-05T00:00:00.000+00:00,2023-08-06T00:00:00.000+00:00,Gift,846313,Concur Campus Card Clearing,Biomedical Informatics,3000025,300000A,Health Science,300200B,School of Medicine,300204C,Medicine,2023
519827,6492602,TE_KRK4_G_LewisLeslie,UCSD Concur,536300 - Gifts/Awards For Non-Employees,505,2024-06-01T03:11:46.000+00:00,2024-06-01T00:00:00.000+00:00,2024-06-02T00:00:00.000+00:00,Gift,1093234,Concur Campus Card Clearing,Urban and Rural,1000170,100000A,Academic Affairs,100600B,Dean of Social Sciences,100607C,Urban and Rural,2024
282474,4813681,TE_FJVH_G_SarikWathnita,UCSD Concur,Gift Cards - HS Payments,2550,2023-05-20T03:18:20.000+00:00,2023-05-20T00:00:00.000+00:00,2023-05-21T00:00:00.000+00:00,Gift,796017,Concur Campus Card Clearing,Medicine Gastroenterology,3000013,300000A,Health Science,300200B,School of Medicine,300204C,Medicine,2023
958,2056734,TE_4FYM_G_EdgeLori,UCSD Concur,Gift Cards - HS Payments,2557.5,2021-09-14T05:56:26.000+00:00,2021-09-14T00:00:00.000+00:00,2021-09-15T00:00:00.000+00:00,Gift,315453,Concur Campus Card Clearing,Medicine Gastroenterology,3000013,300000A,Health Science,300200B,School of Medicine,300204C,Medicine,2021
326354,5173434,TE_G3HV_G_OjedaVictoria,UCSD Concur,Gift Cards - HS Payments,1050,2023-07-20T03:18:35.000+00:00,2023-07-20T00:00:00.000+00:00,2023-07-21T00:00:00.000+00:00,Gift,834943,Concur Campus Card Clearing,School of Public Health Dept,3000191,300000A,Health Science,300700B,School of Public Health,300700C,School of Public Health,2023
223835,4241081,TE_9HLG_G_BenariehTaylor,UCSD Concur,Non-Employee - Gift/Prize/Award,303,2023-02-11T04:10:31.000+00:00,2023-02-11T00:00:00.000+00:00,2023-02-12T00:00:00.000+00:00,Gift,726282,Concur Campus Card Clearing,Communication,1000149,100000A,Academic Affairs,100600B,Dean of Social Sciences,100605C,Communications,2023
489856,6383793,TE_JVNX_G_SanchezJoseline,UCSD Concur,Non-Employee - Gift/Prize/Award,765,2024-04-20T03:20:41.000+00:00,2024-04-20T00:00:00.000+00:00,2024-04-21T00:00:00.000+00:00,Gift,1069108,Concur Campus Card Clearing,Obstetrics Gynecology and Reproductive Sciences,3000087,300000A,Health Science,300200B,School of Medicine,300212C,Obstetrics Gynecology and Reproductive Sciences,2024
346473,5305668,TE_GL47_G_MancanoJennifer,UCSD Concur,Non-Employee - Gift/Prize/Award,985,2023-08-29T03:17:51.000+00:00,2023-08-29T00:00:00.000+00:00,2023-08-30T00:00:00.000+00:00,Gift,859094,Concur Campus Card Clearing,CPEO Events,6000042,600000A,Student Affairs,600100B,Student LIfe,600103C,ArtPower,2023
468606,6219220,TE_JWJR_G_HernandezAlexa,UCSD Concur,Gift Cards - HS Payments,2785,2024-03-21T03:10:54.000+00:00,2024-03-21T00:00:00.000+00:00,2024-03-22T00:00:00.000+00:00,Gift,1015376,Concur Campus Card Clearing,HS Anesthesiology,3000008,300000A,Health Science,300200B,School of Medicine,300202C,Anesthesiology,2024
173678,3766320,TE_7TEP_G_LozanoJohn,UCSD Concur,Employee - Gift/Prize/Award,51,2022-11-05T04:10:26.000+00:00,2022-11-05T00:00:00.000+00:00,2022-11-06T00:00:00.000+00:00,Gift,657197,Concur Campus Card Clearing,Political Science,1000167,100000A,Academic Affairs,100600B,Dean of Social Sciences,100615C,Political Science,2022


In [10]:
oracle_incorrect.shape[0]

3702

**Quantifying Inconsistencies**: Now that we have found out how many inconsistencies we have in our data, the next step is finding out how those inconsistences are distributed through out the data. We can do this by taking each weirdly formatted supplier and seeing how many times they appear, that way we can correct for the common ones and then do away with the uncommon ones.

In [11]:
oracle_incorrect.groupby('Description').count().sort_values('Invoice ID', ascending = False).head(6)

,Invoice ID,Invoice Number,Invoice Source,Invoice Distribution Amount,Invoice Creation Date,Supplier Invoice Date,Payment Date,Payment Method Code,Payment Number,Supplier Name,Financial Unit,Financial Unit Number,Financial Unit Number Level 2,Financial Unit Name Level 2,Financial Unit Number Level 3,Financial Unit Name Level 3,Financial Unit Number Level 4,Financial Unit Name Level 4,Payment Year
Description,,,,,,,,,,,,,,,,,,,
Gift Cards - HS Payments,1468,1468,1468,1468,1468,1468,1468,1468,1468,1468,1468,1468,1468,1468,1468,1468,1468,1468,1468
Employee - Gift/Prize/Award,1102,1102,1102,1102,1102,1102,1102,1102,1102,1102,1102,1102,1102,1102,1102,1102,1102,1102,1102
Non-Employee - Gift/Prize/Award,461,461,461,461,461,461,461,461,461,461,461,461,461,461,461,461,461,461,461
535100 - Human Subjects (Gift Card),242,242,242,242,242,242,242,242,242,242,242,242,242,242,242,242,242,242,242
536200 - Gifts/Awards For Employees,210,210,210,210,210,210,210,210,210,210,210,210,210,210,210,210,210,210,210
536300 - Gifts/Awards For Non-Employees,79,79,79,79,79,79,79,79,79,79,79,79,79,79,79,79,79,79,79


**Splitting the Description Column**: Now that we have found the inconsistencies in our data. We can split the description column into supplier and description. This will allow us to correct those inconsistencies afterwards.

In [14]:
split_oracle = oracle['Description'].str.split(r'\|\|', n=1, expand=True)
split_oracle.columns = ['Description', 'Supplier']
split_oracle.sample(50)

,Description,Supplier
538860,522500 - Printed Items and Other Classroom Aid...,FEDEX OFFICE 800000836
78803,Hotel,Courtyards
267123,Lab Supplies,AMZN MKTP US*HS53E8SN0
379991,Other Minor Equipment <$5K,BATTERY MART
387771,Airfare,Delta Air Lines
331356,Lab Supplies,AMZN MKTP US*ZM5VH5PF3
33777,Transportation Services,LYFT *CANCEL FEE
566461,522401 - Lab Supplies (taxable),AMAZON MKTPL*RJ2QW8PT1
162166,Professional Services,UPWORK -511317700REF
175631,Airline Fees,AGENT FEE 8900824524954


## Cleaning Supplier Names

The next step in this process is cleaning supplier names and making them consistent. This will help make sure that someone tries and find all entries under a certain supplier, they all show up. This will be done by identifying similar strings and then using replace to replace all of the similar strings with one singular string, so as to categorize them as a certain supplier.

**Deleting the Payee**: A lot of entries seem to display the payee within the supplier tag. Since we don't need the payee, this step will involve deleting the payee, and taking only the supplier name.

In [15]:
split_oracle['Supplier'] = split_oracle['Supplier'].apply(lambda x: x.split('||')[-1] if isinstance(x, str) and '||' in x else x)

In [17]:
split_oracle = split_oracle.sort_values('Supplier')

In [18]:
split_oracle[split_oracle["Supplier"].str.contains(r"\|\|", na = False)]

,Description,Supplier


**Finding Suppliers**: This section seeks to find suppliers that need to be replaced by listing suppliers that are repeated more than 20 times. 

In [19]:
counts = split_oracle['Supplier'].value_counts()
entries = counts[counts > 50].index
candidates = split_oracle[split_oracle['Supplier'].isin(entries)].groupby('Supplier').count()
candidates.sample(5)

,Description
Supplier,
UCSD PARKING FLEX,96
SANDIEGOPADRES,138
TRELLO.COM* ATLASSIAN,105
YOGURT WORLD UCSD,69
TST* PACIFIC CAFE & CATER,213


**Suppliers to Replace**: Lets define a dictionary of regex expressions and suppliers we want to capture under said regex expressions.

In [20]:
to_replace = {
    r'MARSHALLS.*':'MARSHALLS',
    r'MARSHALLS INDSTRL HRDWR.*':'MARSHALLS INDSTRL HRDWR',
    r'AMZN MKTP.*':'AMAZON MARKETPLACE',
    r'AMZN MAR.*':'AMAZON MARKETPLACE',
    r'AMZN MKTP US.*':'AMAZON MARKETPLACE',
    r'AMZN MKTPL.*':'AMAZON MARKETPLACE',
    r'AMAZON MKTPL.*':'AMAZON MARKETPLACE',
    r'AMAZON GROCE.*':'AMAZON GROCERY',
    r'.*COSTCO.*':'COSTCO',
    r'AMAZON\.COM.*':'AMAZON.COM',
    r'DROPBOX.*':'DROPBOX',
    r'LYFT.*':'LYFT',
    r'THE HOME DEPOT.*':'HOME DEPOT',
    r'TARGET.*': 'TARGET',
    r'VONS.*': 'VONS',
    r'SLACK.*': 'SLACK',
    r'RALPHS.*': 'RALPHS',
    r'EBAY.*': 'EBAY',
    r'UBER TRIP.*': 'UBER',
    r'UBER\* TRIP.*': 'UBER',
    r'STAPLES.*': 'STAPLES',
    r'PANERA BREAD.*': 'PANERA BREAD',
    r'EMIRATES AI.*': 'EMIRATES',
    r'AGENT FEE.*': 'AGENT FEE',
    r'STARBUCKS.*': 'STARBUCKS',
    r'MICHAELS.*': 'MICHAELS',
    r'BEST BUY.*': 'BEST BUY',
    r'AIRBNB.*': 'AIR BNB',
    r'SOUTHWES.*': 'Southwest Airlines',
    r'IN N OUT BURGER.*': 'IN N OUT BURGER',
    r'AIR CHINA.*': 'AIR CHINA',
    r'GRUBHUB.*': 'GRUBHUB',
    r'BOARD AND BREW.*': 'BOARD AND BREW',
    r'HERTZ.*': 'HERTZ',
    r'.*DOORDASH.*': 'DOORDASH',
    r'LOWES.*': 'LOWES',
    r'APPLEBEES.*': 'APPLEBEES',
    r"MCDONALD'S.*": "MCDONALD'S",
    r'FEDEX.*': 'FEDEX',
    r'WESTJETAIR.*': 'West Jet Air',
    r'CORNER BAKERY CAFE.*': 'CORNER BAKERY CAFE',
    r'OFFICE DEPOT.*': 'OFFICE DEPOT',
    r'WALMART.*': 'WALMART',
    r'WAL-MART.*': 'WALMART',
    r'MARRIOT.*': 'Marriot Hotels',
    r'GOOGLE.*': 'GOOGLE',
    r'CHEVRON.*': 'CHEVRON',
    r"DOMINO'S.*": "DOMINO'S",
    r"LINKEDIN ADS.*": "LINKEDIN",
    r"JERSEY MIKES.*": "JERSEY MIKES",
    r"WM.*": "WALMART",
    r"VRBO.*": "VRBO",
    r'BESTBUY.*': 'BEST BUY',
    r'HOMEDEPOT.*': 'HOME DEPOT',
    r'LABCORP.*': 'LABCORP',
    r'QDI.*': 'QUEST DIAGNOSTICS',
    r'AIRCANADA.*': 'AIR CANADA',
    r'\*FINANCE CHARGE.*': 'FINANCE CHARGE',
    r'.*CAVA.*': 'CAVA',
    r'4IMPRINT.*': '4IMPRINT',
    r'7 ELEVEN.*': '7-ELEVEN',
    r'ACE PARKING.*': 'ACE PARKING',
    r'ADDGENE.*': 'ADDGENE',
    r'ADOBE.*': 'ADOBE',
    r'ALASKA AIR.*': 'Alaska Airlines',
    r'AMTRAK.*':'AMTRAK',
    r'ARCO.*':'ARCO',
    r'ATLASSIAN.*':'ATLASSIAN',
    r'AZENTA.*':'AZENTA',
    r'BIO RAD LABORATORIES.*':'BIO RAD LABORATORIES',
    r'BIORENDER.*':'BIORENDER',
    r'CALENDLY.*':'CALENDLY',
    r'CANVA.*':'CANVA',
    r'CHATGPT.*':'OPEN AI',
    r'CHICK-FIL-A.*':'CHICK-FIL-A',
    r'CHIPOTLE.*':'CHIPOTLE',
    r'COPYRIGHT CLEARANCE.*':'COPYRIGHT CLEARANCE',
    r'CHIPOTLE.*':'CHIPOTLE',
    r'CROUTONSUCSD.*':'CROUTONSUCSD',
    r'CUSTOMINK.*':'CUSTOMINK LLC',
    r'Courtyards.*':'Courtyards',
    r'DKC*DIGI KEY CORP.*':'DKC*DIGI KEY CORP',
    r'DUNKIN.*':'DUNKIN',
    r'EINSTEIN BROS.*':'EINSTEIN BROS BAGELS',
    r'EZCATER.*':'EZCATER',
    r'FACEBK.*':'FACEBK',
    r'FEDEX.*':'FEDEX',
    r'FRONTIER.*':'Frontier Airlines',
    r'HAWAIIAN AI.*':'Hawaiian Airlines',
    r'HILTON.*':'Hilton Hotels',
    r'HYATT.*':'Hyatt Hotels',
    r'IC\*.*':'INSTA CART',
    r'IKEA.*':'IKEA',
    r'IN \*COR.*':'IN *COR SECURITY',
    r'INTEGRATED DNA TECH.*':'INTEGRATED DNA TECH',
    r'JETBLUE.*':'Jetblue',
    r'JOBELEPHANT.*':'JOBELEPHANT',
    r'LA JOLLA SHORES HOTEL.*':'LA JOLLA SHORES HOTEL',
    r'LAZ PARKING.*':'LAZ PARKING',
    r'LENOVO.*':'LENOVO',
    r'LIFETECHCORP.*':'LIFETECHCORP',
    r'LINDE GAS.*':'LINDE GAS & EQUIPMENT INC',
    r'LINDEGE.*':'LINDE GAS & EQUIPMENT INC',
    r'LINKEDIN.*':'LINKEDIN',
    r'Lyft.*':'LYFT',
    r'MAILCHIMP.*':'MAILCHIMP',
    r'MENDOCINO FARM.*':'MENDOCINO FARM',
    r'MICROSOFT.*':'MICROSOFT',
    r'National Car Rental.*':'National Car Rental',
    r'OFFICE DEPOT.*':'OFFICE DEPOT',
    r'OPENAI.*':'OPENAI',
    r"PAPA JOHN'S.*":"PAPA JOHN'S",
    r'PAYPAL.*':'PAYPAL TRANSACTION',
    r'PHILLIPS.*':'PHILLIPS',
    r'PRIMORDIUM LABS.*':'PRIMORDIUM LABS',
    r'RENAISSANCE.*':'RENAISSANCE HOTELS',
    r'REPLICA.*':'REPLICA SAN DIEGO',
    r'SANTA CRUZ BIOTECHNOLOGY.*':'SANTA CRUZ BIOTECHNOLOGY',
    r'SHELL.*':'SHELL',
    r'SHERWIN WILLIAMS.*':'SHERWIN WILLIAMS',
    r'SMART AND FINAL.*':'SMART AND FINAL',
    r'SPARK FUN ELECTRONICS.*':'SPARK FUN ELECTRONICS',
    r'SPRINGHILL SUITES.*':'STRINGHILL SUITES',
    r'SQ *BARWORKS CATERING.*':'BARWORKS CATERING',
    r'SQUARESPACE.*':'SQUARESPACE',
    r'STICKER MULE.*':'STICKER MULE',
    r'TECHNICAL SAFETY SERVICES.*':'TECHNICAL SAFETY SERVICES',
    r'THE UPS STORE.*':'UPS',
    r'ULINE.*':'ULINE',
    r'UPS.*':'UPS',
    r'UPWORK.*':'UPWORK',
    r'Uber.*':'UBER',
}

**The Replacer**: This function will take the above defined dictionary and use it to replace all the values we want it to replace within the dataframe.

In [21]:
def replacer(df, column, replacements):
    for pattern, replacement in replacements.items():
        df[column] = df[column].str.replace(pattern, replacement, regex=True)
    return df

In [22]:
final_oracle = replacer(split_oracle, "Supplier", to_replace)
final_oracle.sample(50)

,Description,Supplier
84861,Food - Dine In/Delivery/Pickup,DIRTY BIRDS BAR AND GR
268841,Airline Fees,AGENT FEE
277106,Other Materials and Supplies,DICKSSPORTINGGOODS.COM
419453,Office Supplies,SLACK
498201,522403 - Marine Supplies (taxable),CVS/PHARMACY #09926
522992,Airline Fees,AGENT FEE
181977,Homeshare - AirBnB / VRBO / Etc.,VRBO
558297,Airline Fees,American Airlines
137433,Mail/Postage/Messenger Services,POSTAL PLUS INC
143640,Conferences and Training (P-Card),PAYPAL TRANSACTION


**Adjusting for Description Inconsistencies**: A lot of the gift card invoices should be under the supplier "Gift Card," but instead are listed just as descriptions with a null value as supplier. This next section of code should work to fix that.

In [23]:
gift_card = [
    'Gift Cards - HS Payments',
    '535100 - Human Subjects (Gift Card)',
    '536200 - Gifts/Awards For Employees',
    '536300 - Gifts/Awards For Non-Employees',
    'Employee - Gift/Prize/Award',
    'Non-Employee - Gift/Prize/Award'
]

In [24]:
final_oracle.loc[final_oracle['Description'].isin(gift_card), 'Supplier'] = 'Gift Card'        

**Supplier Frequency**: Now that we have cleaned out a lot of the common suppliers, lets group by supplier and see what the distribution of suppliers looks like.

In [25]:
supplier_frequency = final_oracle.groupby('Supplier').count()

**Top Suppliers**: What are our top suppliers now?

In [26]:
supplier_frequency.sort_values('Description', ascending = False).head(50)

,Description
Supplier,
AMAZON MARKETPLACE,61951
AGENT FEE,36411
AMAZON.COM,25076
LYFT,15614
PAYPAL TRANSACTION,7054
Southwest Airlines,6426
HOME DEPOT,6321
MCMASTER-CARR,5976
United Airlines,5769


**What Suppliers are billing the most?**:

In [28]:
oracle['Invoice Distribution Amount'] = oracle['Invoice Distribution Amount'].astype(float)
final_oracle['Invoice Distribution Amount'] = oracle['Invoice Distribution Amount']

In [29]:
supplier_total = final_oracle.groupby('Supplier').sum()
supplier_total.sort_values('Invoice Distribution Amount', ascending = False).head(50)

,Invoice Distribution Amount
Supplier,
AMAZON MARKETPLACE,6981027.10
JAX,6731621.86
Residence Inns,6319019.11
VERIZONWRLSS*RTCCR VB,6163805.95
Gift Card,5193442.20
Marriott Hotels,4890366.59
United Airlines,4477189.89
AMAZON.COM,3738252.24
PAYPAL TRANSACTION,3301616.87


## Statistics: How well is the cleaning doing?

### How are we doing in terms of amount of invoices?

In this section I will look solely at the suppliers I have replaced and access what portion of the overall amount of data they make up, so as to access how well the cleaning process is going.

**How many invoices are there?:** Here we will take a look at how many invoices we have total.

In [30]:
num_invoices = supplier_frequency["Description"].sum()
num_invoices

581406

**How many invoices did we take care of?**: Here I will access how many invoices the data cleaning above took care of. 

In [31]:
regex_invoices = final_oracle.loc[final_oracle['Supplier'].isin(to_replace.values())]
total_regex = regex_invoices.shape[0]
total_regex

267088

In [32]:
giftcard_invoices = final_oracle.loc[final_oracle['Description'].isin(gift_card)]
total_giftcard = giftcard_invoices.shape[0]
total_giftcard

4149

In [33]:
total = total_regex + total_giftcard
total

271237

**What percent of invoices did we capture?**: Of the total amount of invoices, what percent is directly accounted for through modification in this notebook?

In [34]:
(total/num_invoices) * 100

46.651909337020946

**What does this mean?**: While this is a low percentage, it is important to remember that a lot of the invoices could have been already grouped properly meaning that there was no cleaning required. This means that the number is likely a lot higher than the 33 percent we are getting from solely the data that I explictly replaced using regex and manual inputation. 

### How are we doing in terms of total spend accounted for?

In this section I will assess how well the cleaning process is working through looking at how much of total spend is accounted for by the cleaned suppliers. This will give us an alternative way of measuring success.

**What is the total spend this month?**: In this section I will be looking at the total spend this month. To do this, I will have to filter out the dataset to only get rows from this month, then total up the "Invoice Distribution Amount.

In [35]:
final_oracle['Payment Date'] = oracle['Payment Date']

In [36]:
final_oracle['Payment Date'] = pd.to_datetime(final_oracle["Payment Date"])

In [37]:
start_date = pd.to_datetime("2023-07-01").tz_localize('UTC')
end_date = pd.to_datetime("2024-06-30").tz_localize('UTC')

this_year = final_oracle[(final_oracle['Payment Date'] >= start_date) & (final_oracle['Payment Date'] <= end_date)]

In [38]:
this_year.head()

,Description,Supplier,Invoice Distribution Amount,Payment Date
417362,Meals & Incidentals,#03 CAP CITY GRANDVIEW,31.78,2023-12-21 00:00:00+00:00
430820,Meals & Incidentals,#03 CAP CITY GRANDVIEW,28.74,2024-01-19 00:00:00+00:00
317243,Other Materials and Supplies,#04 LAKESHORE LEARNING,83.92,2023-07-05 00:00:00+00:00
407391,Other Materials and Supplies,#04 LAKESHORE LEARNING,79.78,2023-12-08 00:00:00+00:00
369236,Other Materials and Supplies,#04 LAKESHORE LEARNING,296.72,2023-10-08 00:00:00+00:00


In [39]:
total_spend = this_year['Invoice Distribution Amount'].sum()
total_spend

95181821.93

**How much spend did we capture**: How much spend did we capture within the invoices cleaned through this notebook?

In [40]:
regex_invoices_spend = sum(this_year.loc[this_year['Supplier'].isin(to_replace.values())]['Invoice Distribution Amount'])
regex_invoices_spend

22066717.75000041

In [41]:
giftcard_invoices_spend = sum(this_year.loc[this_year['Description'].isin(gift_card)]['Invoice Distribution Amount'])
giftcard_invoices_spend

1700909.59

In [42]:
spend_sum = regex_invoices_spend + giftcard_invoices_spend
spend_sum

23767627.34000041

**What percentage of spend did we capture?**: What percent of the total spend is taken care of by invoices that were cleaned using this notebook?

In [43]:
spend_percent = (spend_sum / total_spend) * 100
spend_percent

24.97076317522052

## Exporting our subset of the data

In this section there will be code to export the subsection of the data we worked with in the statistics section. That will allow for further analysis and collaboration. Note: this is not the final export with the data needed for inputation into tableau, but rather a checkpoint export.

In [45]:
final_oracle['Invoice Creation Date'] = pd.to_datetime(oracle["Invoice Creation Date"])

In [46]:
final_oracle['Supplier Invoice Date'] = pd.to_datetime(oracle["Supplier Invoice Date"])

In [47]:
final_oracle['Invoice ID'] = oracle['Invoice ID']
final_oracle[['Invoice Source',
              'Invoice Number', 
              'Payment Method Code', 
              'Payment Number', 
              'Supplier Type',
              'Financial Unit',
              'Financial Unit Number',
              'Financial Unit Number Level 2',
              'Financial Unit Name Level 2',
              'Financial Unit Number Level 3',
              'Financial Unit Name Level 3',
              'Financial Unit Number Level 4',
              'Financial Unit Name Level 4',
              'Payment Year'
             ]] = oracle[['Invoice Source',
              'Invoice Number', 
              'Payment Method Code', 
              'Payment Number', 
              'Supplier Name',
              'Financial Unit',
              'Financial Unit Number',
              'Financial Unit Number Level 2',
              'Financial Unit Name Level 2',
              'Financial Unit Number Level 3',
              'Financial Unit Name Level 3',
              'Financial Unit Number Level 4',
              'Financial Unit Name Level 4',
              'Payment Year']]
final_oracle = final_oracle[['Invoice ID',
                   'Invoice Number', 
                   'Invoice Source', 
                   'Description',
                'Supplier',
                   'Invoice Distribution Amount', 
                   'Invoice Creation Date',
                   'Supplier Invoice Date',
                   'Payment Date',
                   'Payment Method Code',
                   'Payment Number',
                   'Supplier Type',
                   'Financial Unit',
                   'Financial Unit Number',
                   'Financial Unit Number Level 2',
                   'Financial Unit Name Level 2',
                   'Financial Unit Number Level 3',
                   'Financial Unit Name Level 3',
                   'Financial Unit Number Level 4',
                   'Financial Unit Name Level 4',
                   'Payment Year']]

**Adding Card Types**: I will proceed to distinguish if an invoice is from a travel card or a P-Card transaction.

In [48]:
final_oracle['Category'] = final_oracle['Invoice Number'].apply(lambda x: 'Travel' if '_T_' in x else 'Procurement' if '_P_' in x else 'Unknown')

/var/folders/f7/jyxy75m95bdc7p3xqgwf_zyh0000gn/T/ipykernel_23229/554851063.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_oracle['Category'] = final_oracle['Invoice Number'].apply(lambda x: 'Travel' if '_T_' in x else 'Procurement' if '_P_' in x else 'Unknown')


**In Progress**: Working on adding emails and addition information straight from Concur. Don't worry about this section as of now, will be updated in the near future.

In [49]:
expenses = pd.read_csv('Concur Data.csv', encoding='utf-16')
expenses.head()

,TE User ID,TE User Name Full,Expense Report ID,TE User Email Address,Expense Report Type,Expense Report Payment Status,Expense Report Sent for Payment DateTime,Expense Report Key
0,10360455,"Lee, Ellen Eun-Ok",864DDE3F8C2D4ABB8026,eel013@ucsd.edu,(Legacy) Procurement Card Expenses,Sent for Payment,2023-05-20 02:45:04,318573
1,10362988,"Shabkie, Donna Marie",673422E24F5946558728,dshabkie@ucsd.edu,(Legacy) Event Expenses and Other Reimbursements,Sent for Payment,2023-05-05 02:36:57,304767
2,10432699,"Tamrat, Tihut",00B2A02908B34CAABFD0,titamrat@ucsd.edu,(Legacy) Procurement Card Expenses,Sent for Payment,2020-12-05 04:29:53,38493
3,10359672,"Reyes, Rebecca Ellen",7D4A6363B53D4183BF24,rer010@ucsd.edu,(Legacy) Procurement Card Expenses,Sent for Payment,2022-05-24 02:11:08,180972
4,10358900,"Cardenas, Daniel Joseph",F797CDC66AF544AB8A9C,djcardenas@ucsd.edu,(Legacy) Procurement Card Expenses,Sent for Payment,2023-04-26 02:48:15,292489


In [50]:
requests = pd.read_csv('requests.csv', encoding = 'utf-16')
requests.head()

,TE User ID,TE User Name Full,TE Request ID,TE User Email Address,TE Request Type,TE Request Approval Status,TE Request First Approved DateTime
0,10407986,"Engler, Anne-Marie",3TQV,amengler@ucsd.edu,(Legacy) Card & Payment Products Request,Approved,2021-01-21 01:35:33
1,10578843,"Pierce, Meg",6XDC,mcpierce@ucsd.edu,(Legacy) Card & Payment Products Request,Approved,2022-07-07 15:55:12
2,10373170,"Triplett, Cynthia",3U6X,cyc077@ucsd.edu,(Legacy) Card & Payment Products Request,Approved,2021-01-28 22:27:15
3,10417488,"Thompson, Sheri",6KCE,skt007@ucsd.edu,(Legacy) Card & Payment Products Request,Approved,2022-05-02 21:57:06
4,10578843,"Pierce, Meg",6XD9,mcpierce@ucsd.edu,(Legacy) Card & Payment Products Request,Approved,2022-07-07 15:24:48


In [52]:
final_oracle['Extracted_Invoice_Number'] = final_oracle['Invoice Number'].apply(lambda x: x.split('_')[1] if isinstance(x, str) else None)

/var/folders/f7/jyxy75m95bdc7p3xqgwf_zyh0000gn/T/ipykernel_23229/351295459.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_oracle['Extracted_Invoice_Number'] = final_oracle['Invoice Number'].apply(lambda x: x.split('_')[1] if isinstance(x, str) else None)


In [53]:
merged_1_2 = pd.merge(final_oracle, expenses, left_on='Extracted_Invoice_Number', right_on = 'Expense Report ID', how='left')

In [54]:
final_merged = pd.merge(merged_1_2, requests, left_on ='Extracted_Invoice_Number', right_on='TE Request ID', how='left', suffixes = ('','_t3'))


In [55]:
final_merged.columns

Index(['Invoice ID', 'Invoice Number', 'Invoice Source', 'Description',
       'Supplier', 'Invoice Distribution Amount', 'Invoice Creation Date',
       'Supplier Invoice Date', 'Payment Date', 'Payment Method Code',
       'Payment Number', 'Supplier Type', 'Financial Unit',
       'Financial Unit Number', 'Financial Unit Number Level 2',
       'Financial Unit Name Level 2', 'Financial Unit Number Level 3',
       'Financial Unit Name Level 3', 'Financial Unit Number Level 4',
       'Financial Unit Name Level 4', 'Payment Year', 'Category',
       'Extracted_Invoice_Number', 'TE User ID', 'TE User Name Full',
       'Expense Report ID', 'TE User Email Address', 'Expense Report Type',
       'Expense Report Payment Status',
       'Expense Report Sent for Payment DateTime', 'Expense Report Key',
       'TE User ID_t3', 'TE User Name Full_t3', 'TE Request ID',
       'TE User Email Address_t3', 'TE Request Type',
       'TE Request Approval Status', 'TE Request First Approved DateTim

In [56]:
final_merged['TE User ID'] = final_merged['TE User ID'].combine_first(final_merged['TE User ID_t3'])

In [57]:
final_merged['TE User Name Full'] = final_merged['TE User Name Full'].combine_first(final_merged['TE User Name Full_t3'])

In [58]:
final_merged['TE User Email Address'] = final_merged['TE User Email Address'].combine_first(final_merged['TE User Email Address_t3'])

In [59]:
final_merged = final_merged.drop(columns=['TE User ID_t3', 'TE User Name Full_t3', 'TE User Email Address_t3'])

In [60]:
final_merged

,Invoice ID,Invoice Number,Invoice Source,Description,Supplier,Invoice Distribution Amount,Invoice Creation Date,Supplier Invoice Date,Payment Date,Payment Method Code,...,Expense Report ID,TE User Email Address,Expense Report Type,Expense Report Payment Status,Expense Report Sent for Payment DateTime,Expense Report Key,TE Request ID,TE Request Type,TE Request Approval Status,TE Request First Approved DateTime
0,5822802,TE_061C5C8DC5404E51A54F_T_MrozSusan,UCSD Concur,Meals & Incidentals,#03 CAP CITY GRANDVIEW,31.78,2023-12-20 07:11:14+00:00,2023-11-20 00:00:00+00:00,2023-12-21 00:00:00+00:00,TC,...,061C5C8DC5404E51A54F,smmroz@ucsd.edu,(Legacy) Travel Expenses,Sent for Payment,2023-12-20 06:58:58,399234.0,NaN,NaN,NaN,NaN
1,5969721,TE_53521EDEE1DF4778A224_T_GermainBenjamin,UCSD Concur,Meals & Incidentals,#03 CAP CITY GRANDVIEW,28.74,2024-01-18 04:10:19+00:00,2023-09-30 00:00:00+00:00,2024-01-19 00:00:00+00:00,TC,...,53521EDEE1DF4778A224,bgermain@ucsd.edu,(Legacy) Travel Expenses,Sent for Payment,2024-01-18 03:53:47,397506.0,NaN,NaN,NaN,NaN
2,4886767,TE_527F1C82BBF141049C60_P_Bussey-NealVictoria,UCSD Concur,Office Supplies,#04 LAKESHORE LEARNING,268.60,2023-06-03 03:15:50+00:00,2023-04-12 00:00:00+00:00,2023-06-04 00:00:00+00:00,EC,...,527F1C82BBF141049C60,vbusseyneal@ucsd.edu,(Legacy) Procurement Card Expenses,Sent for Payment,2023-06-03 02:40:33,309206.0,NaN,NaN,NaN,NaN
3,4273727,TE_5DAFEC2470084F6CA802_P_Fortune-GradyRegina,UCSD Concur,Other Materials and Supplies,#04 LAKESHORE LEARNING,237.56,2023-02-25 04:06:43+00:00,2023-02-05 00:00:00+00:00,2023-02-26 00:00:00+00:00,EC,...,5DAFEC2470084F6CA802,refortunegrady@ucsd.edu,(Legacy) Procurement Card Expenses,Sent for Payment,2023-02-25 03:35:07,283003.0,NaN,NaN,NaN,NaN
4,3435346,TE_5751852CF4AE420F86DB_P_EdditPaje,UCSD Concur,Office Supplies,#04 LAKESHORE LEARNING,77.57,2022-08-26 03:06:27+00:00,2022-07-28 00:00:00+00:00,2022-08-27 00:00:00+00:00,EC,...,5751852CF4AE420F86DB,epaje@ucsd.edu,(Legacy) Procurement Card Expenses,Sent for Payment,2022-08-26 02:29:07,215365.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581541,6831585,TE_LRN9_G_HaightMarta,UCSD Concur,535100 - Human Subjects (Gift Card),Gift Card,1020.00,2024-08-28 03:11:21+00:00,2024-08-28 00:00:00+00:00,2024-08-29 00:00:00+00:00,Gift,...,NaN,mhaight@ucsd.edu,NaN,NaN,NaN,NaN,LRN9,Gift Cards & Human Subject Payments,Approved,2024-08-21 23:23:59
581542,6831586,TE_LRVT_G_SwanAshley,UCSD Concur,535100 - Human Subjects (Gift Card),Gift Card,5121.00,2024-08-28 03:11:21+00:00,2024-08-28 00:00:00+00:00,2024-08-29 00:00:00+00:00,Gift,...,NaN,arobb@ucsd.edu,NaN,NaN,NaN,NaN,LRVT,Gift Cards & Human Subject Payments,Approved,2024-08-22 20:36:27
581543,6837744,TE_LRWV_G_HillParisa,UCSD Concur,535100 - Human Subjects (Gift Card),Gift Card,260.00,2024-08-29 03:11:17+00:00,2024-08-29 00:00:00+00:00,2024-08-30 00:00:00+00:00,Gift,...,NaN,pmhill@ucsd.edu,NaN,NaN,NaN,NaN,LRWV,Gift Cards & Human Subject Payments,Approved,2024-08-22 21:12:54
581544,6837745,TE_LT3X_G_PatelSean,UCSD Concur,535100 - Human Subjects (Gift Card),Gift Card,3800.00,2024-08-29 03:11:17+00:00,2024-08-29 00:00:00+00:00,2024-08-30 00:00:00+00:00,Gift,...,NaN,svp004@ucsd.edu,NaN,NaN,NaN,NaN,LT3X,Gift Cards & Human Subject Payments,Approved,2024-08-23 15:21:07


In [63]:
final_oracle['Invoice Distribution Amount'].sum()

255770223.4100001

In [64]:
final_merged['Invoice Distribution Amount'].sum()

255770223.4100001

In [65]:
final_merged.to_csv('Concur Final Merged.csv')